In [2]:
%env PGSERVER=pgdb-plastico-dev.postgres.database.azure.com
%env PGDATABASE=plastico-dev
%env PGUSERNAME=writer_user@pgdb-plastico-dev
%env PGPWD=SurfWriter!

env: PGSERVER=pgdb-plastico-dev.postgres.database.azure.com
env: PGDATABASE=plastico-dev
env: PGUSERNAME=writer_user@pgdb-plastico-dev
env: PGPWD=SurfWriter!


In [1]:
# import prerequesite for ai
import json
import requests
import logging
from utils.ai import is_ai_ready,get_prediction,get_json_prediction,get_trash_label,map_label_to_trash_id_PG,get_trash_time_index,get_trash_first_time,get_clean_timed_prediction,get_df_prediction
# import prerequesite for gps
import gpxpy
import gpxpy.gpx
import json
import subprocess
import datetime
from datetime import datetime
from datetime import timedelta
from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform
from tqdm import tqdm
from utils.gps import extract_gpx_from_gopro,parse_gpx,get_gps_point_list,create_time,create_latitude,create_longitude,create_elevation,fill_gps,long_lat_to_shape_point,transform_geo,get_df_trash_gps
# import prerequesite from media
from utils.media import get_media_duration,get_media_fps
# import prerequesite for postgre
import os
import psycopg2
from utils.postgre import get_pg_connection_string,open_pg_connection,close_pg_connection,insert_trash,get_df_data,insert_trash_df

In [3]:
with open('../data/prediction.json') as json_file:
    jsonPrediction = json.load(json_file)

In [4]:
video_name = '28022020_Boudigau_4.MP4'
prediction = jsonPrediction['detected_trash'][0]
media_fps = get_media_fps(f'/tmp/{video_name}')
gpx_path = extract_gpx_from_gopro(f'/tmp/{video_name}')
gpx_data = parse_gpx(gpx_path)
gps_points = get_gps_point_list(gpx_data)
video_duration = get_media_duration(f'/tmp/{video_name}')
video_duration_sup = int(video_duration)+1
gps_points_filled = fill_gps(gps_points,video_duration_sup)
time_index = get_trash_time_index(prediction,media_fps)
# trash_gps
trash_gps = gps_points_filled[time_index]
shape_trash_gps = long_lat_to_shape_point(trash_gps)
geo_2154 = transform_geo(shape_trash_gps)
geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}
# get trash_type from ai module
label = get_trash_label(prediction)
trash_type = map_label_to_trash_id_PG(label)

<_sre.SRE_Match object; span=(2554, 2592), match='Stream #0:3(eng): Data: bin_data (gpmd'>
ffprobe version 4.2.2 Copyright (c) 2007-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.2_2 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --disable-libjack --disable-indev=jack
  liba

In [5]:
import pandas as pd

In [6]:
# A single timed_prediction
timed_prediction = get_clean_timed_prediction(prediction,4)
timed_prediction

{'time_index': 0,
 'frame_to_box': [0.27, 0.12, 0.3, 0.18],
 'id': 0,
 'label': 'fragments',
 'trash_type_id': 4}

In [7]:
# All timed predictions
for prediction in jsonPrediction['detected_trash']:
    print(prediction)
    print(get_clean_timed_prediction(prediction,4))

{'frame_to_box': {'0': [0.27, 0.12, 0.3, 0.18], '1': [0.27, 0.1, 0.3, 0.24], '2': [0.27, 0.09, 0.3, 0.17]}, 'id': 0, 'label': 'fragments'}
{'time_index': 0, 'frame_to_box': [0.27, 0.12, 0.3, 0.18], 'id': 0, 'label': 'fragments', 'trash_type_id': 4}
{'frame_to_box': {'0': [0.28, 0.45, 0.29, 0.48]}, 'id': 1, 'label': 'fragments'}
{'time_index': 0, 'frame_to_box': [0.28, 0.45, 0.29, 0.48], 'id': 1, 'label': 'fragments', 'trash_type_id': 4}
{'frame_to_box': {'3': [0.18, 0.43, 0.2, 0.48]}, 'id': 2, 'label': 'fragments'}
{'time_index': 0, 'frame_to_box': [0.18, 0.43, 0.2, 0.48], 'id': 2, 'label': 'fragments', 'trash_type_id': 4}
{'frame_to_box': {'5': [0.15, 0.15, 0.18, 0.21]}, 'id': 3, 'label': 'fragments'}
{'time_index': 1, 'frame_to_box': [0.15, 0.15, 0.18, 0.21], 'id': 3, 'label': 'fragments', 'trash_type_id': 4}
{'frame_to_box': {'6': [0.32, 0.11, 0.35, 0.17], '8': [0.32, 0.1, 0.34, 0.2]}, 'id': 4, 'label': 'fragments'}
{'time_index': 1, 'frame_to_box': [0.32, 0.11, 0.35, 0.17], 'id': 4

In [8]:
df_predictions = get_df_prediction(jsonPrediction,4)
df_predictions.head()

,time_index,frame_to_box,id,label,trash_type_id
0,0,"[0.27, 0.12, 0.3, 0.18]",0,fragments,4
1,0,"[0.28, 0.45, 0.29, 0.48]",1,fragments,4
2,0,"[0.18, 0.43, 0.2, 0.48]",2,fragments,4
3,1,"[0.15, 0.15, 0.18, 0.21]",3,fragments,4
4,1,"[0.32, 0.11, 0.35, 0.17]",4,fragments,4


In [9]:
time_index = get_trash_time_index(prediction,media_fps)
trash_gps = gps_points_filled[time_index]
shape_trash_gps = long_lat_to_shape_point(trash_gps)
geo_2154 = transform_geo(shape_trash_gps)
geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}

In [10]:
trash_gps = gps_points_filled[time_index]
shape_trash_gps = long_lat_to_shape_point(trash_gps)
geo_2154 = transform_geo(shape_trash_gps)
geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}

In [11]:
def get_df_prediction(json_prediction:dict,media_fps)->pd.DataFrame:
    """Get AI prediction dictionnary as Dataframe

    Arguments:
        json_prediction {dict} -- a full prediction of AI service as JSON dico

    Returns:
       df_prediction -- the AI prediction as a Dataframe
    """
    timed_prediction_list = []
    for prediction in json_prediction['detected_trash']:
        timed_prediction_list.append(get_clean_timed_prediction(prediction,media_fps))
    df_prediction = pd.DataFrame(timed_prediction_list)
    return df_prediction

In [14]:
df_predictions = get_df_prediction(jsonPrediction,4)
df_predictions.dtypes

time_index        int64
frame_to_box     object
id                int64
label            object
trash_type_id     int64
dtype: object

In [15]:
df_trash_gps = get_df_trash_gps(df_predictions,gps_points_filled)
df_trash_gps.head()

,Time,the_geom,Latitude,Longitude,Elevation
0,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
1,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
2,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
3,2020-02-28 11:52:08+00:00,POINT (340988.5069810392 6289760.635096359),43.615857,-1.448492,52.701
4,2020-02-28 11:52:08+00:00,POINT (340988.5069810392 6289760.635096359),43.615857,-1.448492,52.701


In [16]:
df_data = get_df_data(df_predictions,df_trash_gps)

In [42]:
df_data.head()

,time_index,frame_to_box,id,label,trash_type_id,Time,the_geom,Latitude,Longitude,Elevation
0,0,"[0.27, 0.12, 0.3, 0.18]",0,fragments,4,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
1,0,"[0.28, 0.45, 0.29, 0.48]",1,fragments,4,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
2,0,"[0.18, 0.43, 0.2, 0.48]",2,fragments,4,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
3,1,"[0.15, 0.15, 0.18, 0.21]",3,fragments,4,2020-02-28 11:52:08+00:00,POINT (340988.5069810392 6289760.635096359),43.615857,-1.448492,52.701
4,1,"[0.32, 0.11, 0.35, 0.17]",4,fragments,4,2020-02-28 11:52:08+00:00,POINT (340988.5069810392 6289760.635096359),43.615857,-1.448492,52.701


In [18]:
df_data.dtypes

time_index                               int64
frame_to_box                            object
id                                       int64
label                                   object
trash_type_id                            int64
Time             datetime64[ns, SimpleTZ("Z")]
the_geom                                object
Latitude                               float64
Longitude                              float64
Elevation                              float64
dtype: object

In [41]:
data = df_data.iloc[0]

In [40]:
df_data['trash_type_id'] = df_data['trash_type_id'].values.astype(int)

In [20]:
pg_conn_string = get_pg_connection_string()
pg_connection = open_pg_connection(pg_conn_string)
pg_cursor = pg_connection.cursor()

Connection established


In [42]:
data

time_index                                                 0
frame_to_box                         [0.27, 0.12, 0.3, 0.18]
id                                                         0
label                                              fragments
trash_type_id                                              4
Time                               2020-02-28 11:52:07+00:00
the_geom         POINT (340987.7644429434 6289760.231710243)
Latitude                                             43.6159
Longitude                                            -1.4485
Elevation                                             53.203
Name: 0, dtype: object

In [43]:
insert_trash_df(data,pg_cursor,pg_connection)

ProgrammingError: can't adapt type 'numpy.int64'

In [35]:
data['trash_type_id'].values.astype(int)

AttributeError: 'numpy.int64' object has no attribute 'values'

In [33]:
df_data.dtypes

time_index                               int64
frame_to_box                            object
id                                       int64
label                                   object
trash_type_id                            int64
Time             datetime64[ns, SimpleTZ("Z")]
the_geom                                object
Latitude                               float64
Longitude                              float64
Elevation                              float64
dtype: object

In [34]:
for i,row in df_data.iterrows():
    #row_id = insert_trash_df(row,pg_cursor,pg_connection)
    print(type(row['trash_type_id']))
    print(type(row))
    print(row_id)

<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Series'>
56e8dc66-a129-42ce-b98d-9a652edc2111
<class 'int'>
<class 'pandas.core.series.Se